# Exercise 2.4: ORF detection

This exercise was inspired by [Libeskind-Hadas and Bush, *Computing for Biologists*, Cambridge University Press, 2014](https://www.cs.hmc.edu/CFB).

<hr>

**a)** Write a function, `longest_orf()`, that takes a DNA sequence as input and finds the longest open reading frame (ORF) in the sequence (we will not consider reverse complements). A sequence fragment constitutes an ORF if the following are all true.

1. It begins with `ATG`.
2. It ends with any of `TGA`, `TAG`, or `TAA`.
3. The total number of bases is a multiple of 3.

Note that the sequence `ATG` may appear in the middle of an ORF. So, for example,

    GGATGATGATGTAAAAC

has two ORFs, `ATGATGATGTAA` and `ATGATGTAA`. You would return the first one, since it is longer of these two.

*Hint: The statement for this problem is a bit ambiguous as it is written. What other specification might you need for this function?*

**b)** Use your function to find the longest ORF from the section of the *Salmonella* genome we are investigating.

**c)** Write a function that converts a DNA sequence into a protein sequence. You can of course use the `bootcamp_utils` module.

**d)** Translate the longest ORF you generated in part (b) into a protein sequence and perform a [BLAST search](http://blast.ncbi.nlm.nih.gov/). Search for the protein sequence (a blastp query). What gene is it?

**e)** [*Bonus challenge*] Modify your function to return the `n` longest ORFs. Compute the five longest ORFs for the *Salmonella* genome section we are working with. Perform BLAST searches on them. What are they?

<br />

## Solution

<hr>

In [1]:
import re

import bootcamp_utils

**a)** As in the last problem solution, I will again use principles of TDD.

Something was missing in the specification. Namely, what do we do if there are two ORFs of the same longest length? Do we return the first one, second one, or both? I am arbitrarily choosing to return the one with the 3$'$-most starting index.  

Looking ahead to part (e), I will first write a function to return all ORFs that are not entirely included in a longer ORF. For ease of storage and comparison, I will simply store the ORFS as the index of the start of the ORF and the noninclusive index of the last.

Let's now discuss the algorithm we'll use. There are more efficient ways of finding ORFs, but I will choose a very clear way. We'll first find all start codons. For each start codon, we will find the first in-register stop codon. If there is an in-register stop codon, we store this start-stop pair. At the end, we sort them, longest to shortest.

So, we really have three functions we'll use. `find_all_starts(seq)` will return the indices of all start codons in a sequence. `find_next_in_register_stop(seq)` will scan a sequence that starts with `ATG` and return the exclusive final index of the next in register stop codon. In other words, and ORF starting at index `start` is given by

    seq[start : start + find_next_in_register_stop(seq[start :])]

If there is no such codon, `find_next_in_register_stop()` returns `-1`. Finally, `all_orfs(seq)` returns the sorted tuple of 2-tuples containing the start/stop pairs of the ORFs.

I will use TDD principles for designing these functions, writing the test cases first.

In [2]:
def test_find_all_starts():
    assert find_all_starts("") == tuple()
    assert find_all_starts("GGAGACGACGCAAAAC".lower()) == tuple()
    assert find_all_starts("AAAAAAATGAAATGAGGGGGGTATG".lower()) == (6, 11, 22)
    assert find_all_starts("GGATGATGATGTAAAAC".lower()) == (2, 5, 8)
    assert find_all_starts("GGATGCATGATGTAGAAC".lower()) == (2, 6, 9)
    assert find_all_starts("GGGATGATGATGGGATGGTGAGTAGGGTAAG".lower()) == (
        3,
        6,
        9,
        14,
    )
    assert find_all_starts("GGGatgatgatgGGatgGtgaGtagGGACtaaG".lower()) == (
        3,
        6,
        9,
        14,
    )


def test_find_first_in_register_stop():
    assert find_first_in_register_stop("") == -1
    assert find_first_in_register_stop("GTAATAGTGA".lower()) == -1
    assert (
        find_first_in_register_stop("AAAAAAAAAAAAAAATAAGGGTAA".lower()) == 18
    )
    assert find_first_in_register_stop("AAAAAACACCGCGTGTACTGA".lower()) == 21


def test_all_orfs():
    assert all_orfs("") == tuple()
    assert all_orfs("GGAGACGACGCAAAAC") == tuple()
    assert all_orfs("AAAAAAATGAAATGAGGGGGGTATG") == ((6, 15),)
    assert all_orfs("GGATGATGATGTAAAAC") == ((2, 14),)
    assert all_orfs("GGATGCATGATGTAGAAC") == ((6, 15),)
    assert all_orfs("GGGATGATGATGGGATGGTGAGTAGGGTAAG") == ((3, 21),)
    assert all_orfs("GGGatgatgatgGGatgGtgaGtagGGACtaaG") == ((14, 32), (3, 21))

We'll start with the `find_all_starts()` function.

In [3]:
def find_all_starts(seq):
    """Find all start codons in sequence"""
    return None

And now we'll fail the test.

In [4]:
test_find_all_starts()

AssertionError: 

Now we'll write the function. I'll use regular expressions first, but will also code up the function without them.

In [5]:
def find_all_starts(seq):
    """Find the starting index of all start codons in a lowercase seq"""
    # Compile regex for start codons
    regex_start = re.compile('atg')
        
    # Find the indices of all start codons
    starts = []
    for match in regex_start.finditer(seq):
        starts.append(match.start())
        
    return tuple(starts)

And let's see if it passes the tests.

In [6]:
test_find_all_starts()

Yay!  We passed!  However, since we did not learn regular expressions this year, I will write a function that finds all start codons that does not use them.

In [7]:
def find_all_starts(seq):
    """Find the starting index of all start codons in a lowercase seq"""
    # Initialize array of indices of start codons
    starts = []
    
    # Find index of first start codon (remember, find() returns -1 if not found)
    i = seq.find('atg')
    
    # Keep looking for subsequence incrementing starting point of search
    while i >= 0:
        starts.append(i)
        i = seq.find('atg', i + 1)
        
    return tuple(starts)

Now let's test this new `find_all_starts()` function

In [8]:
test_find_all_starts()

It passed! Yay! Note how useful TDD is here. Whenever we try new ways of doing things, we can use the tests to make sure we didn't break anything.

Now, let's move on to the next function, which finds the first in-register stop codon. Again, we fail, and then write the function.

In [9]:
def find_first_in_register_stop(seq):
    return None

test_find_first_in_register_stop()

AssertionError: 

Nice, beautiful failure. Now, we'll write the function and test it. Again, I'll demonstrate the power of the `re` module.

In [10]:
def find_first_in_register_stop(seq):
    """
    Find first stop codon on lowercase seq that starts at an index
    that is divisible by three
    """
    # Compile regexes for stop codons
    regex_stop = re.compile('(taa|tag|tga)')
    
    # Stop codon iterator
    stop_iterator = regex_stop.finditer(seq)

    # Find next stop codon that is in register
    for stop in stop_iterator:
        if stop.end() % 3 == 0:
            return stop.end()
        
    # Return -1 if we failed to find a stop codon
    return -1

And the test...

In [11]:
test_find_first_in_register_stop()

Great! It passes. Now, I'll write it without regular expressions.

In [12]:
def find_first_in_register_stop(seq):
    """
    Find first stop codon on seq that starts at an index
    that is divisible by three
    """

    seq = seq.lower()

    # Scan sequence for stop codon
    i = 0
    while i < len(seq) - 2 and seq[i:i+3] not in ('taa', 'tag', 'tga'):
        i += 3

    # If before end, found codon, return end of codon
    if i < len(seq) - 2:
        return i + 3
    else: # Failed to find stop codon
        return -1

Let's test this function to make sure it works.

In [13]:
test_find_first_in_register_stop()

Passage! Finally, we apply TDD to write `all_orfs()`.

In [14]:
def all_orfs(seq):
    """Return all ORFs of a sequence."""
    # Make sure sequence is all lower case
    seq = seq.lower()
        
    # Find the indices of all start codons
    start_inds = find_all_starts(seq)
        
    # Keep track of stops
    stop_inds = []
        
    # Initialze ORFs.  Each entry in list is [ORF length, ORF start, ORF stop]
    orfs = []
        
    # For each start codon, find the next stop codon in register
    for start in start_inds:
        relative_stop = find_first_in_register_stop(seq[start:])
        
        if relative_stop != -1:
            # Index of stop codon
            stop = start + relative_stop
            
            # If already had stop, a longer ORF contains this one
            if stop not in stop_inds:
                orfs.append((relative_stop, start, stop))
                stop_inds.append(stop)
                        
    # Get sorted list of ORF length
    orfs = sorted(orfs, reverse=True)
    
    # Remove lengths
    for i, orf in enumerate(orfs):
        orfs[i] = (orf[1], orf[2])
    
    return tuple(orfs)

Now for the tests....

In [15]:
test_all_orfs()

Passage!  We have succeed in generating an ordered list of the ORFs.  Now, let's get what the problem specified, the longest ORF.  Of course, we start with writing tests.

In [16]:
def test_longest_orf():
    assert longest_orf("") == ""
    assert longest_orf("GGAGACGACGCAAAAC") == ""
    assert longest_orf("AAAAAAATGAAATGAGGGGGGTATG") == "ATGAAATGA"
    assert longest_orf("GGATGATGATGTAAAAC") == "ATGATGATGTAA"
    assert longest_orf("GGATGCATGATGTAGAAC") == "ATGATGTAG"
    assert longest_orf("GGGATGATGATGGGATGGTGAGTAGGGTAAG") == "ATGATGATGGGATGGTGA"
    assert longest_orf("GGGatgatgatgGGatgGtgaGtagGGACtaaG") == "atgGtgaGtagGGACtaa"

We'll fail them....

In [17]:
def longest_orf(seq):
    return None

test_longest_orf()

AssertionError: 

We'll write our function, and then test it.

In [18]:
def longest_orf(seq):
    """Longest ORF of a sequence."""
    orfs = all_orfs(seq)
    
    if len(orfs) == 0:
        return ''
    else:
        return seq[orfs[0][0]:orfs[0][1]]

In [19]:
test_longest_orf()

Passage! Success! We now have a reliable function for computing the longest ORF.

**b)** We simply use our new function to find the longest ORF of the *Salmonella* sequence.

In [20]:
def read_fasta(filename):
    """Read a sequence in from a FASTA file containing a single sequence.
    
    We assume that the first line of the file is the descriptor and all
    subsequent lines are sequence.    
    """
    with open(filename, 'r') as f:
        # Read in descriptor
        descriptor = f.readline().rstrip()

        # Read in sequence, stripping the whitespace from each line
        seq = ''
        line = f.readline().rstrip()
        while line != '':
            seq += line
            line = f.readline().rstrip()
            
    return descriptor, seq


# Load in Salmonella sequence
descriptor, seq = read_fasta('data/salmonella_spi1_region.fna')

# Compute it
salmonella_orf = longest_orf(seq)

# Look at it
salmonella_orf

'ATGACCAACTACAGCCTGCGCGCACGCATGATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCTATAACGACCTGCAGCGTCAACTGGAAGATGCCGGCGCCAGTATTATTGAACCGCTCGCCGTCTCCAGCGAATATGGTATGAACTTACAAAACCGGGAGTCTATCGGCCAACTTATCAGCGTCCTGCACCGCAGACACTCTGATATTGTGCGGGCGATTTCCGTTTATGACGATCATAACCGTCTGTTTGTAACGTCTAATTTCCATCTGGACCCCTCACAGATGCAGCTTCCCGCCGGAGCGCCGTTTCCACGTCGTCTGAGCGTTGATCGCCACGGCGATATTATGATTCTGCGCACCCCAATTATCTCGGAGAGCTATTCGCCGGACGAGTCAGCGATTGCTGACGCGAAAAATACCAAAAATATGCTGGGGTATGTGGCGCTTGAACTGGATCTCAAGTCGGTCAGGCTACAGCAATACAAAGAGATTTTTATCTCCAGCGTGATGATGCTTTTTTGTATTGGCATTGCGCTGATCTTTGGCTGGCGGCTTATGCGCGATGTCACCGGGCCTATCCGTAATATGGTGAATACCGTTGACCGCATTCGCCGCGGACAACTGGATAGCCGGGTGGAAGGATTTATGCTGGGCGAACTGGATATGCTGAAAAACGGCATTAATTCCATGGCGATGTCGCTTGCCGCCTATCACGAAGAGATGCAGCATAATATCGATCAGGCCACTTCGGACCTGCGTGAAACCCTTGAGCAGATGGAAATCCAAAACGTTGAGCTGGATCTGGCGAAAAAGCGTGCCCAGGAAGCGGCGCGTATTAAGTCGGAGTTCCTGGCGAACATGTCGCACGAACTGCGAACGCCGCTGAACGGCGTCATTGGCTTTACCCGCCTGACATTAAAAACGGAGCTGAATCCCACCCAGCGCGACCATCTGAACACC

**c)** We can use the `codons` dictionary in the `bootcamp_utils` package to do the translation.  With this in hand, we can write our `translate()` function.  We will scan the DNA sequence, generate a list of amino acids, and then join them into a protein sequence.

In [21]:
def translate(seq):
    """Translate a DNA sequence into protein"""
    # Make sure sequence is upper case
    seq = seq.upper()
    
    # Find start codon
    i = 0
    while i < len(seq) + 2 and seq[i:i+3] != 'ATG':
        i += 1

    # Translate until the stop codon or end of string
    prot = []
    while i < len(seq) - 2 and seq[i:i+3] not in ('TAA', 'TGA', 'TAG'):
        prot.append(bootcamp_utils.codons[seq[i:i+3]])
        i += 3

    return ''.join(prot)

**d)** We can now translate the protein

In [22]:
translate(salmonella_orf)

'MTNYSLRARMMILILAPTVLIGLLLSIFFVVHRYNDLQRQLEDAGASIIEPLAVSSEYGMNLQNRESIGQLISVLHRRHSDIVRAISVYDDHNRLFVTSNFHLDPSQMQLPAGAPFPRRLSVDRHGDIMILRTPIISESYSPDESAIADAKNTKNMLGYVALELDLKSVRLQQYKEIFISSVMMLFCIGIALIFGWRLMRDVTGPIRNMVNTVDRIRRGQLDSRVEGFMLGELDMLKNGINSMAMSLAAYHEEMQHNIDQATSDLRETLEQMEIQNVELDLAKKRAQEAARIKSEFLANMSHELRTPLNGVIGFTRLTLKTELNPTQRDHLNTIERSANNLLAIINDVLDFSKLEAGKLILESIPFPLRNTLDEVVTLLAHSSHDKGLELTLNIKNDVPDNVIGDPLRLQQVITNLVGNAIKFTESGNIDILVEKRALSNTKVQIEVQIRDTGIGIPERDQSRLFQAFRQADASISRRHGGTGLGLVITQKLVNEMGGDISFHSQPNRGSTFWFHINLDLNPNVIIDGPSTACLAGKRLAYVEPNATAAQCTLDLLSDTPVEVVYSPTFSALPLAHYDIMILSVPVTFREPLTMQHERLAKAASMTDFLLLALPCHAQINAEKLKQGGAAACLLKPLTSTRLLPALTEYCQLNHHPEPLLMDTSKITMTVMAVDDNPANLKLIGALLEDKVQHVELCDSGHQAVDRAKQMQFDLILMDIQMPDMDGIRACELIHQLPHQQQTPVIAVTAHAMAGQKEKLLSAGMNDYLAKPIEEEKLHNLLLRYKPGANVAARLMAPEPAEFIFNPNATLDWQLALRQAAGKPDLARDMLQMLIDFLPEVRNKIEEQLVGENPNGLVDLVHKLHGSCGYSGVPRMKNLCQLIEQQLRSGVHEEELEPEFLELLDEMDNVAREAKKILG'

Doing a BLAST search on this protein indicates that it is a histidine kinase involved in signaling.

**e)** We already are computing all of the ORFs. We an therefore just add a kwarg to our `longest_orf()` function to get the `n` longest ones.

In [23]:
def longest_orf(seq, n=1):
    """Longest ORF of a sequence."""
    orfs = all_orfs(seq)
    
    if len(orfs) == 0:
        return ''
    elif n == 1 or len(orfs) == 1:
        return seq[orfs[0][0]:orfs[0][1]]
    else:
        return_list = []
        for i in range(min(n, len(orfs))):
            return_list.append(seq[orfs[i][0]:orfs[i][1]])
            
        return tuple(return_list)

Now, we'll compute the ORFs, translate them, and make a FASTA file to submit for a BLAST search.

In [24]:
# Compute ORFs
orfs = longest_orf(seq, n=5)

# Translate them
prots = []
for orf in orfs:
    prots.append(translate(orf))
    
# Make a FASTA file
with open('sal_seqs.faa', 'w') as f:
    for i, prot in enumerate(prots):
        f.write('> {0:d}\n'.format(i))
        f.write(prot + '\n')

We can take a look at it to see what I did with the above code.

In [25]:
!cat sal_seqs.faa

> 0
MTNYSLRARMMILILAPTVLIGLLLSIFFVVHRYNDLQRQLEDAGASIIEPLAVSSEYGMNLQNRESIGQLISVLHRRHSDIVRAISVYDDHNRLFVTSNFHLDPSQMQLPAGAPFPRRLSVDRHGDIMILRTPIISESYSPDESAIADAKNTKNMLGYVALELDLKSVRLQQYKEIFISSVMMLFCIGIALIFGWRLMRDVTGPIRNMVNTVDRIRRGQLDSRVEGFMLGELDMLKNGINSMAMSLAAYHEEMQHNIDQATSDLRETLEQMEIQNVELDLAKKRAQEAARIKSEFLANMSHELRTPLNGVIGFTRLTLKTELNPTQRDHLNTIERSANNLLAIINDVLDFSKLEAGKLILESIPFPLRNTLDEVVTLLAHSSHDKGLELTLNIKNDVPDNVIGDPLRLQQVITNLVGNAIKFTESGNIDILVEKRALSNTKVQIEVQIRDTGIGIPERDQSRLFQAFRQADASISRRHGGTGLGLVITQKLVNEMGGDISFHSQPNRGSTFWFHINLDLNPNVIIDGPSTACLAGKRLAYVEPNATAAQCTLDLLSDTPVEVVYSPTFSALPLAHYDIMILSVPVTFREPLTMQHERLAKAASMTDFLLLALPCHAQINAEKLKQGGAAACLLKPLTSTRLLPALTEYCQLNHHPEPLLMDTSKITMTVMAVDDNPANLKLIGALLEDKVQHVELCDSGHQAVDRAKQMQFDLILMDIQMPDMDGIRACELIHQLPHQQQTPVIAVTAHAMAGQKEKLLSAGMNDYLAKPIEEEKLHNLLLRYKPGANVAARLMAPEPAEFIFNPNATLDWQLALRQAAGKPDLARDMLQMLIDFLPEVRNKIEEQLVGENPNGLVDLVHKLHGSCGYSGVPRMKNLCQLIEQQLRSGVHEEELEPEFLELLDEMDNVAREAKKILG
> 1
MNESFDKDFSNHTPMMQQYLKLKAQHPEILLFYRMGDFYELFYDDAKRASQLLDISLTKRGASAGEPIPMAGI

Upon doing the BLAST search, I found that the genes are:

|Length rank| Description|
|:---:|:---:|
|1 | histine kinase |
|2 | DNA repair protein MutS|
|3 | formate hydrogenlyase transcriptional activator|
|4 | L-fucose isomerase |
|5 | transcriptional regulator HilA (invasion regulator)|

## Computing environment

In [26]:
%load_ext watermark
%watermark -v -p re,bootcamp_utils,jupyterlab

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.3.0

re            : 2.2.1
bootcamp_utils: 0.0.6
jupyterlab    : 3.3.2

